In [1]:
import numpy as np
from transformers import (BertTokenizerFast, BertForTokenClassification)
import torch
from sklearn import metrics

In [2]:
def TransformarTextoEmInput(textos):
    tokenizados = []
    for indice in range(len(textos)):
            tokens = tokenizer.encode_plus(textos[indice], add_special_tokens=True, truncation=True, return_tensors='pt')
            tokens = tokens['input_ids']
            #tokens = tokens.type(torch.int64).to(device)
            tokenizados.append(tokens)
    return tokenizados

def TransformarNotasEmVetor(textos, notas):
    novas_notas = []
    for indice in range(len(textos)):
            novas_notas.append(torch.tensor(notas[indice]).unsqueeze(0))
    return novas_notas

In [3]:
def le_dataset_argumentos(pasta):
    import os
    lista_argumentos = []
    lista_notas = []
    path = ".\\Argumentos-e-notas\\"+pasta+"\\"
    tamanho = len(os.listdir(path)) //2
    for i in range(tamanho):
        file_arg = open(path+str(i)+"A.txt","r",encoding='utf-8')
        file_nota = open(path+str(i)+"N.txt","r")
        lista_notas.append(float(file_nota.read()))
        args = file_arg.read()
        args = args.replace("\n"," [SEP] ")
        lista_argumentos.append(args)
        file_nota.close()
        file_arg.close()
    return lista_argumentos, lista_notas    


def filtrar_dataset(argumentos, notas, num_maximo):
    contador = [0]*6
    argumentos_filtrados = []
    notas_filtradas = []
    for arg, nota in zip(argumentos, notas):
        if contador[int(nota)] < num_maximo:
            argumentos_filtrados.append(arg)
            notas_filtradas.append(nota)
            contador[int(nota)] += 1
        #else:
            #print("filtrado")
    return argumentos_filtrados, notas_filtradas

def filtrar_dataset2(argumentos, notas, num_maximo, desejados):
    argumentos_filtrados = []
    notas_filtradas = []
    contador = [0]*2
    for arg, nota in zip(argumentos, notas):
        if int(nota) in desejados:
            argumentos_filtrados.append(arg)
            if nota == 4.0:
                notas_filtradas.append(1.0)
            else:
                notas_filtradas.append(0.0)
            contador[0] += 1
    return argumentos_filtrados, notas_filtradas

argumentos_treinamento, nota_treinamento = le_dataset_argumentos("Treinamento")
#argumentos_treinamento, nota_treinamento = filtrar_dataset2(argumentos_treinamento, nota_treinamento, 300, [0,4])
argumentos_validacao, nota_valid = le_dataset_argumentos("Validacao")
#argumentos_validacao, nota_valid = filtrar_dataset2(argumentos_validacao, nota_valid, 300, [0,4])

In [5]:
import torch.nn as nn
from transformers import AutoModel
from transformers import AutoTokenizer


class CustomModel(nn.Module):
    def __init__(self): 
        super(CustomModel,self).__init__() 
        self.model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
        #self.dropout = nn.Dropout(0.1) 
        self.classifier = nn.Sequential(nn.Linear(768,1020), nn.Tanh(), nn.Linear(1020,560), nn.Tanh(), nn.Linear(560,6), nn.Tanh(), nn.Linear(6,6), nn.Softmax(0))
        #self.classifier = nn.Sequential(nn.Linear(768,12), nn.Tanh(), nn.Linear(12,1))
    def forward(self, input_ids):
        
        #input_ids = input_ids.unsqueeze(0)
        #print("Input: ", input_ids.size())
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids)
            outputs = outputs['pooler_output']
            #print(outputs.size())
        logits = self.classifier(outputs) 
        return logits
tokenizer_bertimbau = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased',model_max_length=512, padding='max_length', truncation=True, do_lower_case=False)

In [6]:
def argumentos2inputs(lista_lista_argumentos):
    nova_lista = []
    for lista in lista_lista_argumentos:
        tokenizado = tokenizer_bertimbau(lista, padding='max_length')
        nova_lista.append(torch.tensor(tokenizado['input_ids']))
    print(len(nova_lista), len(lista_lista_argumentos))
    assert len(nova_lista) == len(lista_lista_argumentos)
    return nova_lista

def treinar(model, inputs, target):
    #loss_fn = torch.nn.MSELoss()
    loss_fn = nn.CrossEntropyLoss()
    #print(target)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    i = 0
    vetor_loss = []
    for batch, target in zip(inputs,target):
        #i += 1
        #print(i)
        batch = batch.to("cuda")
        #print("Target antes:", target)
        target = torch.tensor(target).long().to("cuda")
        optimizer.zero_grad()
        output = model(batch)
       # print("Output: ", output.size())
       # print("Target: ", target.size(), target)
        #output = output.squeeze(1)
        loss = loss_fn(output, target )
        vetor_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        batch.cpu().detach()
        target.cpu().detach()
    print("Loss media: ", sum(vetor_loss)/len(vetor_loss))

def acuracia_classe(respostas, gold_labels):
    classes_chutadas = [0]*6
    classes_acertadas = [0]*6
    classes_certas = [0]*6
    porcentagem_final = []
    for r, gl in zip(respostas, gold_labels):
        #print(gl)
        classes_certas[gl] += 1
        classes_chutadas[r] += 1
        if (r == gl):
            classes_acertadas[r] += 1
    for acertos, certo in zip(classes_acertadas, classes_certas):
        if certo != 0:
            porcentagem_final.append( float(acertos) / certo )
        else:
            porcentagem_final.append(0.0)       
    print("Chutei: ", classes_chutadas, sum(classes_chutadas))
    #print("Acertei: ", classes_acertadas, sum(classes_acertadas))
    print("Dist: ", classes_certas, sum(classes_certas))
    return porcentagem_final

    
#acuracia_classe([5, 5, 5, 5], [5, 5, 5, 5])
melhor_qwk = -1
melhor_epoca = -1
def testar(model, inputs, target, epoca):
    global melhor_qwk, melhor_epoca
    respostas = []
    target = np.array(target).flatten().astype(int)
    #print(target)
    with torch.no_grad():
        for batch in inputs:
            batch = batch.to("cuda")
            output = model(batch)
            nota = output.cpu().detach().numpy()
            #nota_final = np.rint(nota)
            nota_final = np.argmax(nota,axis=1)
            respostas.append(nota_final)
            batch.cpu().detach()
            #target.cpu().detach()
    respostas = np.array(respostas).flatten().astype(int)
    #print(respostas.size, target.size )
    #print(target)
    QWK = metrics.cohen_kappa_score(target, respostas, weights='quadratic')
    print("QWK: ", QWK)
    #print("RMSE: ", metrics.mean_squared_error(target, respostas, squared=False))
    print("MSE: ", metrics.mean_squared_error(target, respostas, squared=True))
    print("Acc: ", metrics.accuracy_score(target, respostas))
    print("Porcentagem das classes: ", acuracia_classe(respostas, target))
    if (QWK > melhor_qwk):
        melhor_qwk = QWK
        melhor_epoca = epoca
    print(f"Estou na epoca {epoca} e a melhor QWK foi {melhor_qwk} na epoca {melhor_epoca}")

In [7]:
def criar_batch_size(lista_tensores, lista_notas, tamanho):
    inicio = 0
    fim = tamanho
    nova_lista = []
    novas_notas = []
    while( (inicio < len(lista_tensores)) and (fim < len(lista_tensores))):
        stack = lista_tensores[inicio:fim]
        notas = lista_notas[inicio:fim]
        nova_lista.append(torch.stack(stack))
        novas_notas.append(notas)
        inicio += tamanho
        fim += tamanho
    return nova_lista, novas_notas


torch.cuda.empty_cache()
t = argumentos2inputs(argumentos_treinamento)
t2 = argumentos2inputs(argumentos_validacao)
batch_size = 8
t, nota_treinamentoN = criar_batch_size(t, nota_treinamento, batch_size)
t2, nota_validN = criar_batch_size(t2, nota_valid, batch_size)
t2 = torch.stack(t2)
print(len(t))

3198 3198
686 686
399


In [ ]:
model2 = CustomModel().to("cuda")
for i in range(30):
    print("Epoca: ", i+1)
    print("  Treinamento:")
    treinar(model2, t, nota_treinamentoN)
    #print("   Teste no treinamento:")
    #testar(model2, t, nota_treinamentoN, i)
    print("  Validacao:")
    testar(model2, t2, nota_validN, i)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoca:  1
  Treinamento:
Loss media:  1.772725583019113
  Validacao:
QWK:  0.4177427068388331
MSE:  1.4323529411764706
Acc:  0.38235294117647056
Chutei:  [8, 110, 42, 71, 444, 5] 680
Dist:  [19, 13, 101, 266, 242, 39] 680
Porcentagem das classes:  [0.05263157894736842, 0.3076923076923077, 0.10891089108910891, 0.13533834586466165, 0.859504132231405, 0.0]
Estou na epoca 0 e a melhor QWK foi 0.4177427068388331 na epoca 0
Epoca:  2
  Treinamento:
Loss media:  1.7669126252482708
  Validacao:
QWK:  0.38760631834750925
MSE:  1.4823529411764707
Acc:  0.38676470588235295
Chutei:  [24, 61, 79, 65, 449, 2] 680
Dist:  [19, 13, 101, 266, 242, 39] 680
Porcentagem das classes:  [0.05263157894736842, 0.23076923076923078, 0.2079207920792079, 0.12030075187969924, 0.8512396694214877, 0.0]
Estou na epoca 1 e a melhor QWK foi 0.4177427068388331 na epoca 0
Epoca:  3
  Treinamento:
Loss media:  1.763606510066747
  Validacao:
QWK:  0.34649239797279274
MSE:  1.4411764705882353
Acc:  0.40588235294117647
Chutei:

In [ ]:
#torch.cuda.empty_cache()
#torch.cuda.memory_summary(device=None, abbreviated=False)